# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 17 2022 4:02PM,237476,10,0085679042,ISDIN Corporation,Released
1,May 17 2022 4:02PM,237476,10,0085679043,ISDIN Corporation,Released
2,May 17 2022 4:02PM,237476,10,0085679067,ISDIN Corporation,Released
3,May 17 2022 4:02PM,237476,10,0085679095,ISDIN Corporation,Released
4,May 17 2022 4:02PM,237476,10,0085679096,ISDIN Corporation,Released
5,May 17 2022 4:02PM,237476,10,0085679093,ISDIN Corporation,Released
6,May 17 2022 4:02PM,237476,10,0085679094,ISDIN Corporation,Released
7,May 17 2022 4:02PM,237476,10,0085679103,ISDIN Corporation,Released
8,May 17 2022 4:02PM,237476,10,0085679105,ISDIN Corporation,Released
9,May 17 2022 4:02PM,237476,10,0085679104,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
62,237471,Released,1
63,237473,Released,1
64,237474,Released,1
65,237475,Released,2
66,237476,Released,10


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237471,NaN,NaN,1.0
237473,NaN,NaN,1.0
237474,NaN,NaN,1.0
237475,NaN,NaN,2.0
237476,NaN,NaN,10.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237023,0.0,1.0,0.0
237381,0.0,1.0,0.0
237382,0.0,0.0,1.0
237384,0.0,0.0,56.0
237385,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237023,0,1,0
237381,0,1,0
237382,0,0,1
237384,0,0,56
237385,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237023,0,1,0
1,237381,0,1,0
2,237382,0,0,1
3,237384,0,0,56
4,237385,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237023,,1,
1,237381,,1,
2,237382,,,1
3,237384,,,56
4,237385,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 17 2022 4:02PM,237476,10,ISDIN Corporation
10,May 17 2022 3:58PM,237474,16,Sartorius Stedim Filters Inc.
11,May 17 2022 3:57PM,237475,10,Emerginnova
13,May 17 2022 3:55PM,237473,10,Beach Patient Assistance
14,May 17 2022 3:52PM,237449,18,"Blinc International, Inc.-Blinc-ecom"
15,May 17 2022 3:52PM,237450,18,"Blinc International, Inc.-Blinc-ecom"
16,May 17 2022 3:52PM,237451,18,"Blinc International, Inc.-Blinc-ecom"
17,May 17 2022 3:51PM,237452,18,"Blinc International, Inc.-Blinc-ecom"
18,May 17 2022 3:51PM,237453,18,"Blinc International, Inc.-Blinc-ecom"
19,May 17 2022 3:51PM,237454,18,"Blinc International, Inc.-Blinc-ecom"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 17 2022 4:02PM,237476,10,ISDIN Corporation,,,10
1,May 17 2022 3:58PM,237474,16,Sartorius Stedim Filters Inc.,,,1
2,May 17 2022 3:57PM,237475,10,Emerginnova,,,2
3,May 17 2022 3:55PM,237473,10,Beach Patient Assistance,,,1
4,May 17 2022 3:52PM,237449,18,"Blinc International, Inc.-Blinc-ecom",,,1
5,May 17 2022 3:52PM,237450,18,"Blinc International, Inc.-Blinc-ecom",,,1
6,May 17 2022 3:52PM,237451,18,"Blinc International, Inc.-Blinc-ecom",,,1
7,May 17 2022 3:51PM,237452,18,"Blinc International, Inc.-Blinc-ecom",,,1
8,May 17 2022 3:51PM,237453,18,"Blinc International, Inc.-Blinc-ecom",,,1
9,May 17 2022 3:51PM,237454,18,"Blinc International, Inc.-Blinc-ecom",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 4:02PM,237476,10,ISDIN Corporation,10,,
1,May 17 2022 3:58PM,237474,16,Sartorius Stedim Filters Inc.,1,,
2,May 17 2022 3:57PM,237475,10,Emerginnova,2,,
3,May 17 2022 3:55PM,237473,10,Beach Patient Assistance,1,,
4,May 17 2022 3:52PM,237449,18,"Blinc International, Inc.-Blinc-ecom",1,,
5,May 17 2022 3:52PM,237450,18,"Blinc International, Inc.-Blinc-ecom",1,,
6,May 17 2022 3:52PM,237451,18,"Blinc International, Inc.-Blinc-ecom",1,,
7,May 17 2022 3:51PM,237452,18,"Blinc International, Inc.-Blinc-ecom",1,,
8,May 17 2022 3:51PM,237453,18,"Blinc International, Inc.-Blinc-ecom",1,,
9,May 17 2022 3:51PM,237454,18,"Blinc International, Inc.-Blinc-ecom",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 4:02PM,237476,10,ISDIN Corporation,10,,
1,May 17 2022 3:58PM,237474,16,Sartorius Stedim Filters Inc.,1,,
2,May 17 2022 3:57PM,237475,10,Emerginnova,2,,
3,May 17 2022 3:55PM,237473,10,Beach Patient Assistance,1,,
4,May 17 2022 3:52PM,237449,18,"Blinc International, Inc.-Blinc-ecom",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 4:02PM,237476,10,ISDIN Corporation,10.0,NaN,NaN
1,May 17 2022 3:58PM,237474,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
2,May 17 2022 3:57PM,237475,10,Emerginnova,2.0,NaN,NaN
3,May 17 2022 3:55PM,237473,10,Beach Patient Assistance,1.0,NaN,NaN
4,May 17 2022 3:52PM,237449,18,"Blinc International, Inc.-Blinc-ecom",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 4:02PM,237476,10,ISDIN Corporation,10.0,0.0,0.0
1,May 17 2022 3:58PM,237474,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
2,May 17 2022 3:57PM,237475,10,Emerginnova,2.0,0.0,0.0
3,May 17 2022 3:55PM,237473,10,Beach Patient Assistance,1.0,0.0,0.0
4,May 17 2022 3:52PM,237449,18,"Blinc International, Inc.-Blinc-ecom",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3324019,120.0,25.0,7.0
15,237441,0.0,1.0,0.0
16,712247,3.0,0.0,0.0
18,5698565,25.0,1.0,0.0
19,949727,25.0,4.0,0.0
20,3086418,74.0,2.0,0.0
21,237381,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3324019,120.0,25.0,7.0
1,15,237441,0.0,1.0,0.0
2,16,712247,3.0,0.0,0.0
3,18,5698565,25.0,1.0,0.0
4,19,949727,25.0,4.0,0.0
5,20,3086418,74.0,2.0,0.0
6,21,237381,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,120.0,25.0,7.0
1,15,0.0,1.0,0.0
2,16,3.0,0.0,0.0
3,18,25.0,1.0,0.0
4,19,25.0,4.0,0.0
5,20,74.0,2.0,0.0
6,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,120.0
1,15,Released,0.0
2,16,Released,3.0
3,18,Released,25.0
4,19,Released,25.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,7.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,25.0,1.0,0.0,1.0,4.0,2.0,1.0
Released,120.0,0.0,3.0,25.0,25.0,74.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,7.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,25.0,1.0,0.0,1.0,4.0,2.0,1.0
2,Released,120.0,0.0,3.0,25.0,25.0,74.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,7.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,25.0,1.0,0.0,1.0,4.0,2.0,1.0
2,Released,120.0,0.0,3.0,25.0,25.0,74.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()